<a href="https://colab.research.google.com/github/MaguireMaName/Coursera_Capstone/blob/master/The_Battle_of_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone: Battle of the Neighbourhoods

##Introduction/Business Problem

This project will bring together neighboured information on venues (from Foursquare) and neighbourhood crime statistics to find complementary neighbourhood concerning local venues, but to compare and contrast alike neighbourhoods with crime statistics.

This information will inform a business who wishes to set up shop in a reasonably safe neighbourhood or enable an individual to identify a venue to visit in a reasonably safe neighbourhood.

## Data description

The Foursquare data will be used to identify similar neighbourhoods by the frequency of venues by type. This alikeness will allow businesses to identify those neighbourhoods with relatively high levels of competition. For example, if I wanted to set up a coffee shop, I might want to avoid a neighbourhood saturated with coffee shops!

Furthermore, the value of the Foursquare dataset is to be extended by the inclusion of local crime dataset alongside resident population dataset. The integration of these datasets will enable business or individuals to identify further those venues and neighbourhoods that intersect with unreasonably high incidences of crime on a per capita basis.

This information will enable business or individuals to identify on a map those neighbourhoods based on the type of venues and the incidence of crime per capita.  Allowing businesses to target and assess the potential location of their operations by frequency of venues (i.e., competition) and low crime per capita.  Similarly, individuals can use the information to assess what venues to visit in what neighbourhood by type of venue and the incidence of crime in that neighbourhood.

## Methodology

### 1.  Install/import dependencies

In [0]:
#!pip install geocoder

In [0]:
# load dependancies
import pandas as pd 
import numpy as np
from numpy import *
import geocoder
import folium
from folium import plugins
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

### 2.  Import the neighbourhood dataset and inspect its dimensions

In [11]:
cbr = pd.read_csv('Canberra suburbs.csv') # read canberra neighbourhoods
print(cbr.shape) # check dimensions
cbr.head() # inspect data

(124, 4)


,Neighborhood,Postcode,Country,Region
0,ACTON,2601,Australia,Australian Capital Territory
1,AINSLIE,2602,Australia,Australian Capital Territory
2,AMAROO,2914,Australia,Australian Capital Territory
3,ARANDA,2614,Australia,Australian Capital Territory
4,BANKS,2906,Australia,Australian Capital Territory


### 3.  Geocode the neighbourhood dataset

In [12]:
Lat_list=[]
Lng_list=[]

for i in range(cbr.shape[0]):
    address='{}, Canberra, Australia'.format(cbr.at[i,'Neighborhood']) # geocoding each canberra neighbourhood
    g = geocoder.arcgis(address)
    Lat_list.append(g.latlng[0])
    Lng_list.append(g.latlng[1])
    
for i in range(cbr.shape[0]):  
  
    cbr['Latitude'] = Lat_list   # add the lat to Canberra neighbourhood dataset 
    cbr['Longitude'] = Lng_list  # add the lng to Canberra neighbourhood dataset 
    
print(cbr.shape) # check dimensions
cbr.head() # inspect data

(124, 6)


,Neighborhood,Postcode,Country,Region,Latitude,Longitude
0,ACTON,2601,Australia,Australian Capital Territory,-35.28562,149.11827
1,AINSLIE,2602,Australia,Australian Capital Territory,-35.26222,149.14655
2,AMAROO,2914,Australia,Australian Capital Territory,-35.16922,149.12637
3,ARANDA,2614,Australia,Australian Capital Territory,-35.25804,149.08293
4,BANKS,2906,Australia,Australian Capital Territory,-35.47004,149.09771


### 4.  Visualise the point data of neighbourhoods

In [33]:
address = 'Canberra, Australian Capital Territory'

geolocator = Nominatim(user_agent="canberra_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_cbr = folium.Map(location=[latitude, longitude], zoom_start=11, tiles='Stamen Toner') # create a base map of Canberra using latitude and longitude values

# add markers to map
for lat, lng, label in zip(cbr['Latitude'], cbr['Longitude'], cbr['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7.5,
        tooltip=label,
        popup=label,
        color='pink',
        fill=True,
        fill_color='blue',
        fill_opacity=0.8,
        parse_html=False).add_to(map_cbr)  
    
map_cbr

In [0]:
# initialise Foursquare credentials, version no., and limt

client_id = 'KL5SVGOS40RKZBQK4G1VXYBKBICWCDQL2NMCASHFYER432SS' 
client_secret = '1A5KPYJQIATH0SDZXPPZ5YK0SHLBYVEGPER5AAIIMDXLZ0AB'
version = '20180604'
limit = 100

In [0]:
# create a function to lookup venues and iterate across in Canberra neighborhoods dataset

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # create the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:
# run the above function on each neighborhood and create a new dataframe called Canberra venues

cbr_venues = getNearbyVenues(names=cbr['Neighborhood'],
                                   latitudes=cbr['Latitude'],
                                   longitudes=cbr['Longitude']
                                  )

ACTON
AINSLIE
AMAROO
ARANDA
BANKS
BARTON
BEARD
BELCONNEN
BLACK MOUNTAIN
BONNER
BONYTHON
BRADDON
BRUCE
CALWELL
CAMPBELL
CAPITAL HILL
CASEY
CHAPMAN
CHARNWOOD
CHIFLEY
CHISHOLM
CITY
CONDER
COOK
COOMBS
CRACE
CURTIN
DEAKIN
DENMAN PROSPECT
DICKSON
DOWNER
DUFFY
DUNLOP
DUNTROON
EVATT
FADDEN
FARRER
FISHER
FLOREY
FLYNN
FORDE
FORREST
FRANKLIN
FRASER
FYSHWICK
GARRAN
GILMORE
GIRALANG
GORDON
GOWRIE
GREENWAY
GRIFFITH
GUNGAHLIN
HACKETT
HALL
HARMAN
HARRISON
HAWKER
HIGGINS
HOLDER
HOLT
HUGHES
HUME
ISAACS
ISABELLA PLAINS
JACKA
KALEEN
KAMBAH
KENNY
KINGSTON
KINLYSIDE
LATHAM
LAWSON
LYNEHAM
LYONS
MACARTHUR
MACGREGOR
MACQUARIE
MAJURA
MAWSON
MCKELLAR
MELBA
MITCHELL
MOLONGLO
MONASH
MONCRIEFF
NARRABUNDAH
NGUNNAWAL
NICHOLLS
OAKS ESTATE
O'CONNOR
O'MALLEY
OXLEY
PAGE
PALMERSTON
PARKES
PEARCE
PHILLIP
PIALLIGO
RED HILL
REID
RICHARDSON
RIVETT
RUSSELL
SCULLIN
SPENCE
STIRLING
STROMLO
SYMONSTON
TAYLOR
THARWA
THEODORE
THROSBY
TORRENS
TURNER
URIARRA
WANNIASSA
WARAMANGA
WATSON
WEETANGERA
WESTON
WILLIAMSDALE
WRIGHT
YARRALUMLA


In [39]:


print(cbr_venues.shape) # check dimensions
cbr_venues.head() # inspect data

(663, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ACTON,-35.28562,149.11827,Monster Kitchen and Bar,-35.285122,149.122547,Hotel Bar
1,ACTON,-35.28562,149.11827,Palace Electric Cinema,-35.285014,149.123135,Movie Theater
2,ACTON,-35.28562,149.11827,National Film & Sound Archive,-35.283131,149.121143,Museum
3,ACTON,-35.28562,149.11827,University House Hotel,-35.283743,149.118193,Hotel
4,ACTON,-35.28562,149.11827,The Fellows Bar & Cafe,-35.283781,149.117636,Bar


### Aggregate venues over each neighborhood

In [0]:

cbr_venues.groupby('Neighborhood').count() # aggregate the number of venues returned for each neighborhood

print('There are {} unique venue categories.'.format(len(cbr_venues['Venue Category'].unique())))
print(cbr_venues.head())

There are 165 unique venue categories.
  Neighborhood  Neighborhood Latitude  ...  Venue Longitude         Venue Category
0        ACTON              -35.28562  ...       149.122547              Hotel Bar
1        ACTON              -35.28562  ...       149.123135          Movie Theater
2        ACTON              -35.28562  ...       149.121143                 Museum
3        ACTON              -35.28562  ...       149.117795  Australian Restaurant
4        ACTON              -35.28562  ...       149.116116                   Café

[5 rows x 7 columns]


In [40]:
# analyse each neighbourhood
cbr_onehot = pd.get_dummies(cbr_venues[['Venue Category']], prefix="", prefix_sep="") # one hot encoding

cbr_onehot['Neighborhood'] = cbr_venues['Neighborhood'] # add neighborhood column back to dataframe

fixed_columns = [cbr_onehot.columns[-1]] + list(cbr_onehot.columns[:-1]) # move neighborhood column to the first column 
cbr_onehot = cbr_onehot[fixed_columns]

cbr_onehot.head()

,Neighborhood,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Bakery,Bar,Baseball Field,Beer Bar,Bike Trail,Bistro,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Campaign Office,Cantonese Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Construction & Landscaping,Convenience Store,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,...,Record Shop,Recreation Center,Resort,Restaurant,River,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Soccer Field,Social Club,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,ACTON,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,ACTON,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,ACTON,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,ACTON,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,ACTON,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
cbr_grouped = cbr_onehot.groupby('Neighborhood').mean().reset_index()
cbr_grouped

,Neighborhood,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Bakery,Bar,Baseball Field,Beer Bar,Bike Trail,Bistro,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Campaign Office,Cantonese Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Construction & Landscaping,Convenience Store,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,...,Record Shop,Recreation Center,Resort,Restaurant,River,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Soccer Field,Social Club,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,ACTON,0.0,0.00,0.00,0.000000,0.000000,0.111111,0.0,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.222222,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,...,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.000000
1,AINSLIE,0.0,0.00,0.00,0.142857,0.142857,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,...,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000,0.0,0.000000,0.000000,0.142857,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.000000
2,AMAROO,0.0,0.00,0.50,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,...,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.000000
3,ARANDA,0.0,0.00,0.00,0.000000,0.000000,0.166667,0.0,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.166667,0.000,0.000000,0.000000,0.000000,0.0,...,0.00,0.166667,0.00,0.00,0.00,0.000000,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.000000
4,BANKS,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,...,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.000000
5,BARTON,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.041667,0.00,0.000000,0.041667,0.0

In [42]:
# top 10 frequencies

num_top_venues = 10

for hood in cbr_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = cbr_grouped[cbr_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ACTON----
                 venue  freq
0        Movie Theater  0.22
1                 Café  0.22
2  Indie Movie Theater  0.11
3                  Bar  0.11
4            Hotel Bar  0.11
5                Hotel  0.11
6               Museum  0.11
7  Arts & Crafts Store  0.00
8    Other Repair Shop  0.00
9            Nightclub  0.00


----AINSLIE----
                     venue  freq
0           Shopping Plaza  0.14
1                      Pub  0.14
2    Australian Restaurant  0.14
3                   Bakery  0.14
4        Fish & Chips Shop  0.14
5                     Café  0.14
6            Grocery Store  0.14
7      Arts & Crafts Store  0.00
8             Noodle House  0.00
9  North Indian Restaurant  0.00


----AMAROO----
                     venue  freq
0       Athletics & Sports   0.5
1                 Pharmacy   0.5
2      Arts & Crafts Store   0.0
3        Other Repair Shop   0.0
4                   Museum   0.0
5              Music Venue   0.0
6                Nightclub   0.0
7    

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = cbr_grouped['Neighborhood']

for ind in np.arange(cbr_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cbr_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ACTON,Café,Movie Theater,Bar,Museum,Hotel Bar,Indie Movie Theater,Hotel,Hostel,Furniture / Home Store,Fruit & Vegetable Store
1,AINSLIE,Café,Pub,Australian Restaurant,Bakery,Shopping Plaza,Fish & Chips Shop,Grocery Store,Food & Drink Shop,Furniture / Home Store,Fruit & Vegetable Store
2,AMAROO,Athletics & Sports,Pharmacy,Yoga Studio,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
3,ARANDA,Café,Gift Shop,Recreation Center,Bar,Dance Studio,Food & Drink Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint
4,BANKS,Grocery Store,Yoga Studio,Garden,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court,Food & Drink Shop


In [45]:
# cluster neighbourhoods

# set number of clusters
kclusters = 4

cbr_grouped_clustering = cbr_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cbr_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5]

array([0, 0, 0, 0, 3], dtype=int32)

In [0]:
# add clustering labels
neighborhoods_venues_sorted.insert(1, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.tail()

In [48]:
# merge dataset and check output

cbr_merged = pd.merge(cbr, neighborhoods_venues_sorted, on='Neighborhood')
cbr_merged.tail()

,Neighborhood,Postcode,Country,Region,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
104,WANNIASSA,2903,Australia,Australian Capital Territory,-35.40288,149.09194,0,Ice Cream Shop,Theater,Sandwich Place,Pool,Hostel,Filipino Restaurant,Fried Chicken Joint,French Restaurant,Food Court,Food & Drink Shop
105,WARAMANGA,2611,Australia,Australian Capital Territory,-35.35331,149.06015,3,Soccer Field,Grocery Store,Yoga Studio,Food & Drink Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
106,WATSON,2602,Australia,Australian Capital Territory,-35.24132,149.15731,3,Café,Grocery Store,Shopping Plaza,Filipino Restaurant,Fish & Chips Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
107,WESTON,2611,Australia,Australian Capital Territory,-35.33660,149.05535,0,Café,Fast Food Restaurant,Sandwich Place,Cricket Ground,Farmers Market,Food Court,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop
108,WRIGHT,2611,Australia,Australian Capital Territory,-35.32441,149.04089,0,River,Yoga Studio,Flower Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court,Food & Drink Shop


In [168]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11, tiles='Stamen Toner')

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cbr_merged['Latitude'], cbr_merged['Longitude'], cbr_merged['Neighborhood'], cbr_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [53]:
# cluster 0
cbr_merged.loc[cbr_merged['Cluster Labels'] == 0, cbr_merged.columns[[1] + list(range(5, cbr_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2601,149.118270,0,Café,Movie Theater,Bar,Museum,Hotel Bar,Indie Movie Theater,Hotel,Hostel,Furniture / Home Store,Fruit & Vegetable Store
1,2602,149.146550,0,Café,Pub,Australian Restaurant,Bakery,Shopping Plaza,Fish & Chips Shop,Grocery Store,Food & Drink Shop,Furniture / Home Store,Fruit & Vegetable Store
2,2914,149.126370,0,Athletics & Sports,Pharmacy,Yoga Studio,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
3,2614,149.082930,0,Café,Gift Shop,Recreation Center,Bar,Dance Studio,Food & Drink Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint
5,2600,149.133540,0,Café,Hotel,Coffee Shop,Bistro,Event Space,French Restaurant,Cantonese Restaurant,Sports Bar,Breakfast Spot,Gym
6,2620,149.212170,0,Home Service,Dance Studio,Yoga Studio,Flower Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
7,2617,149.063450,0,Coffee Shop,Café,Music Venue,Gym,Fast Food Restaurant,Electronics Store,Ice Cream Shop,Burrito Place,Liquor Store,Multiplex
8,2601,149.097540,0,Mountain,Monument / Landmark,History Museum,Scenic Lookout,Flower Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
9,2905,149.080200,0,Wine Shop,IT Services,Cupcake Shop,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
10,2612,149.138330,0,Café,Hotel,Pizza Place,Australian Restaurant,Pub,Molecular Gastronomy Restaurant,Food & Drink Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop


In [54]:
# cluster 1
cbr_merged.loc[cbr_merged['Cluster Labels'] == 1, cbr_merged.columns[[1] + list(range(5, cbr_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,2906,149.08503,1,Bus Station,Spa,Yoga Studio,Food & Drink Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
50,2914,149.16094,1,Bus Station,Park,Yoga Studio,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
72,2617,149.07570,1,Bus Station,Japanese Restaurant,Yoga Studio,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
73,2615,149.05312,1,Bus Station,Yoga Studio,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
77,2914,149.11349,1,Bus Station,Yoga Studio,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
83,2903,149.08206,1,Bus Station,Yoga Studio,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
85,2913,149.11758,1,Bus Station,Grocery Store,Yoga Studio,Food & Drink Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
102,2607,149.08789,1,Bus Station,Shop & Service,Yoga Studio,Food & Drink Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court


In [55]:
# cluster 2
cbr_merged.loc[cbr_merged['Cluster Labels'] == 2, cbr_merged.columns[[1] + list(range(5, cbr_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,2612,149.16005,2,Bus Stop,Yoga Studio,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
27,2615,149.02221,2,Bus Stop,Yoga Studio,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
76,2904,149.09226,2,Bus Stop,Yoga Studio,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court


In [57]:
# cluster 3
cbr_merged.loc[cbr_merged['Cluster Labels'] == 3, cbr_merged.columns[[1] + list(range(5, cbr_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,2906,149.09771,3,Grocery Store,Yoga Studio,Garden,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court,Food & Drink Shop
11,2617,149.09268,3,Café,Flower Shop,Pub,Grocery Store,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
15,2611,149.04076,3,Bus Station,Grocery Store,Track,Yoga Studio,Food & Drink Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
20,2614,149.06442,3,Café,Bike Trail,Grocery Store,Gym,Chinese Restaurant,Fast Food Restaurant,Food Court,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store
22,2911,149.10606,3,IT Services,Gastropub,Park,Grocery Store,Flower Shop,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
29,2617,149.07286,3,Bakery,Fast Food Restaurant,Shopping Plaza,Grocery Store,Yoga Studio,Food & Drink Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint
33,2615,149.05536,3,Fish & Chips Shop,Pizza Place,Pharmacy,Shopping Plaza,Grocery Store,Fish Market,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
40,2605,149.10875,3,Malay Restaurant,Bakery,Restaurant,Grocery Store,Fish Market,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
43,2904,149.10654,3,Café,Pharmacy,Shopping Mall,Gym / Fitness Center,Grocery Store,Event Space,Food & Drink Shop,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint
47,2602,149.16391,3,Mountain,Grocery Store,Trail,Café,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant


### Import Canberra crime statisitcs

In [170]:
# read in canberra neighbourhoods

cbr_crime = pd.read_excel('Canberra Neighborhoods Crime Stats.xlsx')
print(cbr_crime.shape) # check dimensions
cbr_crime.head() # inspect data

(124, 7)


,Neighborhood,Homicide,Assault,Robbery,Burglary,Total,Population
0,ACTON,4,23,2,21,50,2277
1,AINSLIE,0,37,3,57,97,5313
2,AMAROO,0,33,1,36,70,5849
3,ARANDA,0,12,3,18,33,2500
4,BANKS,0,10,1,25,36,4924


In [184]:
cbr_crime['Total per pop'] = (cbr_crime['Total'] /  cbr_crime['Population']) * 100
cbr_crime['Rank'] = cbr_crime['Total per pop'].rank(ascending=1)
cbr_crime = pd.merge(cbr_merged, cbr_crime, on='Neighborhood').sort_values('Rank')
print(cbr_crime.shape) # check dimensions
cbr_crime.head() # inspect data

(105, 94)


,Neighborhood,Postcode_x,Country_x,Region_x,Latitude_x,Longitude_x,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postcode_y,Country_y,Region_y,Latitude_y,Longitude_y,Cluster Labels_x,1st Most Common Venue_x,2nd Most Common Venue_x,3rd Most Common Venue_x,4th Most Common Venue_x,5th Most Common Venue_x,6th Most Common Venue_x,7th Most Common Venue_x,8th Most Common Venue_x,9th Most Common Venue_x,10th Most Common Venue_x,Postcode_x,Country_x,Region_x,Latitude_x,Longitude_x,Cluster Labels_y,1st Most Common Venue_y,...,Cluster Labels_x,1st Most Common Venue_x,2nd Most Common Venue_x,3rd Most Common Venue_x,4th Most Common Venue_x,5th Most Common Venue_x,6th Most Common Venue_x,7th Most Common Venue_x,8th Most Common Venue_x,9th Most Common Venue_x,10th Most Common Venue_x,Postcode_x,Country_x,Region_x,Latitude_x,Longitude_x,Cluster Labels_y,1st Most Common Venue_y,2nd Most Common Venue_y,3rd Most Common Venue_y,4th Most Common Venue_y,5th Most Common Venue_y,6th Most Common Venue_y,7th Most Common Venue_y,8th Most Common Venue_y,9th Most Common Venue_y,10th Most Common Venue_y,Postcode_y,Country_y,Region_y,Latitude_y,Longitude_y,Homicide,Assault,Robbery,Burglary,Total,Population,Total per pop,Rank
27,DUNTROON,2600,Australia,Australian Capital Territory,-35.30000,149.16667,0,Café,Golf Course,Sports Club,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,2600,Australia,Australian Capital Territory,-35.30000,149.16667,0,Café,Golf Course,Sports Club,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,2600,Australia,Australian Capital Territory,-35.30000,149.16667,0,Café,...,0,Café,Golf Course,Sports Club,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,2600,Australia,Australian Capital Territory,-35.30000,149.16667,0,Café,Golf Course,Sports Club,Food & Drink Shop,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,2600,Australia,Australian Capital Territory,-35.30000,149.16667,0,0,0,0,0,1700,0.000000,1.5
72,MOLONGLO,2911,Australia,Australian Capital Territory,-35.33333,149.18333,0,Café,Furniture / Home Store,Pharmacy,Other Repair Shop,Sporting Goods Shop,Electronics Store,Hockey Arena,History Museum,Frozen Yogurt Shop,Fried Chicken Joint,2911,Australia,Australian Capital Territory,-35.33333,149.18333,0,Café,Furniture / Home Store,Pharmacy,Other Repair Shop,Sporting Goods Shop,Electronics Store,Hockey Arena,History Museum,Frozen Yogurt Shop,Fried Chicken Joint,2911,Australia,Australian Capital Territory,-35.33333,149.18333,0,Café,...,0,Café,Furniture / Home Store,Pharmacy,Other Repair Shop,Sporting Goods Shop,Electronics Store,Hockey Arena,History Museum,Frozen Yogurt Shop,Fried Chicken Joint,2911,Australia,Australian Capital Territory,-35.33333,149.18333,0,Café,Furniture / Home Store,Pharmacy,Other Repair Shop,Sporting Goods Shop,Electronics Store,Hockey Arena,History Museum,Frozen Yogurt Shop,Fried Chicken Joint,2911,Australia,Australian Capital Territory,-35.33333,149.18333,0,0,0,0,0,376,0.000000,1.5
45,GUNGAHLIN,2912,Australia,Australian Capital Territory,-35.18537,149.13728,0,Café,Coffee Shop,Supermarket,Liquor Store,Pub,Department Store,Gym,Japanese Restaurant,Fried Chicken Joint,Sandwich Place,2912,Australia,Australian Capital Territory,-35.18537,149.13728,0,Café,Coffee Shop,Supermarket,Liquor Store,Pub,Department Store,Gym,Japanese Restaurant,Fried Chicken Joint,Sandwich Place,2912,Australia,Australian Capital Territory,-35.18537,149.13728,0,Café,...,0,Café,Coffee Shop,Supermarket,Liquor Store,Pub,Department Store,Gym,Japanese Resta

In [180]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11, tiles='Stamen Toner')

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cbr_crime['Latitude_x'], cbr_crime['Longitude_x'], cbr_crime['Neighborhood'], cbr_crime['Cluster Labels'], cbr_crime['Rank'] ):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

TypeError: ignored